# Import packages
Note: additional python packages may need to be installed in your environment

In [ ]:
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels
import array
import datetime
import time
import requests as req

from datetime import timedelta
import sys
from sqlalchemy.engine import URL, create_engine

from IPython.display import clear_output
pd.options.mode.chained_assignment = None  # default='warn'

# Functions for Event Processing
The event processing uses the following steps:
1. reduce_columns: takes a dataframe and reduces it to only the necessary columns
2. group_data: groups points on a dataframe into an energy level and a duration
3. smooth_data: smooths out data to produce a dataframe where the lifetimeEnergy is always increasing
4. calculate_events: calculates events using a dataframe where hte lifetimeEnergy is always increasing
5. data_quality_check: indicates how much energy was captured in the event definition
6. data_qualiy_control: adds in unaccounted for energy into events
7. adjust_time: corrects the timezone
8. add_meter_ids: adds in the meterIds based on a fetch_list or other lookup table

Of these, the smooth_data, and calculate_events are the most complex/critical and the others are simple helper functions.

The smooth_data function operates with the understanding that the lifetimeEnergy values from the smart meters should *always* be increasing (this is true after grouping datapoints with the same lifetimeEnergy value into a single datapoint). The function extracts and orders the unique lifetimeEnergy values from a dataframe. Then, it searches the dataframe for datapoints where the lifetimeEnergy value is out of order and corrects the value based on the neighboring values and expected values from the ordered list. It iterates through this process, expected to take less than 10 iterations for a given meter.

The calculate_events function takes a dataset and identifies the starts and ends of events based on a time duration criteria (default 15 minutes). In the most basic case, if the lifetimeEnergy does not increase for the specified duration then that datapoint signals the start and the end of an event. Additionally, a gap between datapoints of more than the specified duration would also signal an event start and end. More nuance is added to address situations where multiple datapoints in a row meet the criteria for an event start or end.

There are likely ways to improve the efficiency and clarity of these two functions, such as by refining both functions to involve fewer specific criteria ("if it looks like this, this should happen").

### Basic Instructions (tl;dr):
1. Run all of the following cells to define the functions
2. Go to the "Try it Out" section

In [ ]:
def reduce_columns(df):
    df = df.loc[:, ("account_id", "time", "lifetimeEnergy")]
    return df

In [ ]:
def group_data(df, raw = True):
    """
    This function is for grouping data according to the lifetimeEnergy. For "raw" data, it does an additional sorting step
    and then aggregates the time column, which is then renamed to measurementTime_min and _max columns. The raw == False
    loop is used in the smoothing function for data that has already been grouped before.
    """
    if raw == True:
        clear_output(wait=True)
        print("Grouping data...")
        # sort the data by the account and then the measurement time
        df = df.sort_values(['account_id','time']).fillna(method = 'ffill').reset_index()
    
        # Add a column that enables us to group consecutive rows. This is later used in the smoothing 
        # function as it allows us to differentiate groups of the same lifetimeEnergy values.
        df["grouping"] = df["lifetimeEnergy"].diff().ne(0).cumsum()
    
        # Then group the data to simplify it into a new structure: start, end, energy value
        df = df.groupby(["account_id", "grouping", "lifetimeEnergy"], as_index=False)\
        .aggregate(measurementTime_min = ('time', 'min'),
                   measurementTime_max = ('time', 'max'),
                  )
        
    else:
        # add a column that enables us to group consecutive rows
        df["grouping"] = df["lifetimeEnergy"].diff().ne(0).cumsum()

        df = df.groupby(["account_id", "grouping", "lifetimeEnergy"], as_index=False)\
        .aggregate(measurementTime_min = ('measurementTime_min', 'min'),
                   measurementTime_max = ('measurementTime_max', 'max'),
                  )
    
    return df

In [ ]:
def smooth_data(df, col = "lifetimeEnergy"):
    """
    This function smooths smart meter data so that it is monotonically (always) increasing. 
    It consists of a smoothing function called smoother and two loops.
    
    This function is much more complicated than intended. However, after resolving back-end issues, it effects very little.
    """
    
    
    def smoother(df, col):
        """
        First step: create columns to help identify where the data needs to be corrected
        
        pv_e: the lifetimeEnergy value that is expected to preced the current datapoint
        fv_e: the lifetimeEnergy value that is expected to follow the current datapoint
        pv: 0 if the preceding lifetimeEnergy value is incorrect, 1 if it is correct
        fv: 0 if the following lifetimeEnergy value is incorrect, 1 if it is correct
        ev: a single digit representation of whether a point is ordered correctly based on the preceding and following values
        """
        # Create a temporary column used to store updated values
        df["temp"] = df[col]
        
        # Create an ordered list of the unique energy values
        # For easy access, add two columns stating the expected preceding and following values (pv_e and fv_e)
        y_ordered = df.loc[:, [col]].sort_values(col).drop_duplicates()
        y_ordered["pv_e"] = y_ordered["lifetimeEnergy"].shift(1)
        y_ordered["fv_e"] = y_ordered["lifetimeEnergy"].shift(-1)
        
        # Merge the dataframe with the ordered values
        df = df.merge(y_ordered, left_on = col, right_on = col).sort_values(by = 'grouping')
        
        # Create a mask for rows where the expected preceding value matches the actual preceding value
        # Set pv = 1 if the preceding value is as expected based on the ordered list
        mask_pv = df["pv_e"] == df[col].shift(1)
        df["pv"] = 0
        df["pv"] = df["pv"].mask(mask_pv, 1)
        
        # Create a mask for rows where the expected following value matches the actual following value
        # Set fv = 2 if the following value is as expected based on the ordered list
        mask_fv = df["fv_e"] == df[col].shift(-1)
        df["fv"] = 0
        df["fv"] = df["fv"].mask(mask_fv, 2)
        
        # Create a new column for the "expected value"
        # 0: both the preceding and following values are incorrect
        # 1: only the preceding value is correct
        # 2: only the following value is correct
        # 3: both the preceding and following values are correct
        df["ev"] = df["pv"]+df["fv"]
        
        
        """
        Step 2: Correct the incorrectly ordered points based on the surrounding datapoints.
        """
        
        # Find the places where:
        # 1: the preceding row has a correct preceding value but incorrect following value (ev == 1)
        # 2: and where the following row has a correct following value but incorrect preceding value (ev == 2)
        # 3: and where the following row has a value greater than or equal to the value of the preceding row
        #
        # Set the lifetimeEnergy value to be the average of the preceding and following lifetimeEnergy values.
        #
        # ex: Evaluating the third value in this series (lifetimeEnergy == 10)
        # lifetimeEnergy:    [12, 13, 10, 14, 15]  goes to [12, 13, 13.5, 14, 15]
        #             ev:    (x,   1,  0,  2,  x)
        df["temp"].loc[(df["ev"].shift(1) == 1) & (df["ev"].shift(-1) == 2) & (df[col].shift(-1) >= df[col].shift(1))
                      ] = (df["lifetimeEnergy"].shift(1) + df["lifetimeEnergy"].shift(-1)) / 2
        
        
        # If the third condition above is false (i.e. its a downward slope), instead change the preceding and 
        # following rows and set them to be equal to the their preceding/following rows
        #
        # ex: evaluating the second value in this series (lifetimeEnergy == 13) and the fourth value (lifetimeEnergy == 12) 
        #     [12, 13, 10, 12, 13] goes to [12, 12, 10, 13, 13] which is resolved on the next Iteration
        #     ( x,  1,  2,  3,  x)
        #
        # adjusting the second value
        df["temp"].loc[(df["ev"] == 1) & (df["ev"].shift(-2) == 2) & (df[col].shift(-2) < df[col])
                      ] = df[col].shift(1)
        # adjusting the fourth value
        df["temp"].loc[(df["ev"].shift(2) == 1) & (df["ev"] == 2) & (df[col] < df[col].shift(2))
                      ] = df[col].shift(-1)
        
        
        # Find the places where:
        # 1: the current row has a correct preceding value but incorrect following value (ev == 1)
        # 2: the following row has a correct following value but incorrect preceding value (ev == 2)
        #
        # Assign the value of the current row to the value preceding it, and
        # assign the value of the following row to the one following that one
        # ex: [12, 13, 12, 13] goes to [12, 12, 13, 13]
        #     ( x,  1,  2,  x)
        df["temp"].loc[(df["ev"] == 1) & (df["ev"].shift(-1) == 2)
                      ] = df[col].shift(1)
        df["temp"].loc[(df["ev"] == 2) & (df["ev"].shift(1) == 1)
                      ] = df[col].shift(-1)
        
        # Find the places where:
        # 1: the preceding row has both incorrect preceding and following values (ev == 1)
        # 2: the preceding row has a correct preceding value but incorrect following value (ev == 0)
        # 3: the following row has incorrect preceding and following values (ev == 0)
        #
        # Set the current row to the expected following value of the preceding row
        # ex: [12, 13, 17, 12, 14] goes to [12, 13, 14, 12, 14] 
        #     ( x,  1,  0,  0,  x)         ( x,  3,  1,  0,  x)
        df["temp"].loc[(df["ev"] == 0) & (df["ev"].shift(1) == 1) & (df["ev"].shift(-1) == 0)
                      ] = df["fv_e"].shift(1)
        
        # Find the places where:
        # 1: the preceding row has both incorrect preceding and following values (ev == 0)
        # 2: the current row has incorrect preceding and following values (ev == 0)
        # 3: the following row has a correct following value but incorrect preceding value (ev == 2)
        #
        # Set the current row to the preceding value that is expected from the following row
        # ex: [12, 11, 17, 13, 14] goes to [12, 11, 12, 13, 14] 
        #     ( x,  0,  0,  2,  x)         ( x,  2,  3,  3,  x)
        df["temp"].loc[(df["ev"] == 0) & (df["ev"].shift(1) == 0) & (df["ev"].shift(-1) == 2)
                      ] = df["pv_e"].shift(-1)
        
        # Find the places where:
        # 1: the preceding row has a correct preceding value but incorrect following value (ev ==1 )
        # 2: the current row has incorrect preceding and following values (ev == 0)
        # 3: the following row and preceding rows have the same lifetimeEnergy values
        #
        # Set the current row to the value before/after
        # ex: [12, 13, 17, 13, 14] goes to [12, 13, 13, 18, 13]
        #     (x,   1,  3,  x,  x) 
        df["temp"].loc[(df["ev"].shift(2) == 3) & (df["ev"].shift(1) == 1) & (df[col].shift(1) == df[col].shift(-1))
                      ] = df[col].shift(1)
        
        # Round the values to two decimal points
        df[col] = df["temp"].round(2)
        
        # Fix the first and last values
        df[col].iat[0] = df[col].min()
        df[col].iat[-1] = df[col].max()
        
        df = group_data(df, raw = False)
        return df
    
    df_out = pd.DataFrame()
    
    # Main loop for running the smoother for each meter
    # m: variable just for tracking how many meters have been completed
    # i: variable for tracking iterations. Passes into the smoother function
    m = 0
    for account in df["account_id"].unique():
        m = m+1
        i = 1
        
        # First pull the data from an individual meter
        df_i = df.loc[df["account_id"] == account]
        
        # The main loop for iterating through each meter
        # The .is_monotonic_increasing function results in the need for iterating through meters individually
        while df_i[col].is_monotonic_increasing == False:
            # run the smoothing function
            df_i = smoother(df_i, col)
            
            # update progress and timeout as necessary
            clear_output(wait=True)
            print("Processing account ID " + str(account) + ", estimated progress = " + str(round(100 * (m-1) / len(df["account_id"].unique()),2)) + "%")
            i = i + 1
            if i == 50:
                print("Error: timeout")
                return df_i
        
        # add the individual meter's smoothed data to the output
        print("Concatenating Data")
        df_out = pd.concat([df_out, df_i])
    
    return df_out

In [ ]:
def calculate_events(df, ycol="lifetimeEnergy", minutes = 15, strict = "on"):
    clear_output(wait=True)
    print("Calculating events...")
    """
    Calculate the time difference between points
    Just group by the meter & energy and aggregate time
    If the time at a given value is greater than the minutes variable, then remove those values, but keep the head and tail
    """
    
    # calculate the difference in time between subsequent datapoints
    df['measurementTime_min'] = pd.to_datetime(df['measurementTime_min'])
    df['measurementTime_max'] = pd.to_datetime(df['measurementTime_max'])
    
    # the 'backward' time difference is the difference between a datapoint and the previous one
    # the backward time difference signals the start of an event: more than 15 minutes have passed since the last value
    df["time_diff_b"] = df['measurementTime_min'] - df['measurementTime_max'].shift(1, fill_value = df['measurementTime_max'].min() - timedelta(minutes=100))
    df["time_diff_b"].loc[df["account_id"] != df["account_id"].shift(1)] = timedelta(minutes=100)
    df["time_diff_b"] = df["time_diff_b"] / timedelta(minutes=1)
    #return df
    
    # the 'forward' time difference is the difference between a datapoint and the following value
    # this signals the end of a cooking event
    df["time_diff_f"] = df['measurementTime_min'].shift(-1, fill_value = df['measurementTime_min'].max() + timedelta(minutes=100)) - df['measurementTime_max']
    df["time_diff_f"].loc[df["account_id"] != df["account_id"].shift(-1)] = timedelta(minutes=100)
    df["time_diff_f"] =  df["time_diff_f"] / timedelta(minutes=1)
    
    df["duration"] = df['measurementTime_max'] - df['measurementTime_min']
    df["duration"] = df["duration"] / timedelta(minutes=1)

    # group the data: find the highest and lowest measurement times at each energy value per meter
    # and sum the forward and backward differences
    df_group = df.groupby(["account_id", ycol], as_index=False).aggregate(measurementTime_min = ('measurementTime_min', 'min'),
                                                              measurementTime_max = ('measurementTime_max', 'max'),
                                                              time_diff_b = ('time_diff_b', 'sum'),
                                                              time_diff_f = ('time_diff_f', 'sum'),
                                                                          duration = ("duration", 'sum')
                                                             )
    
    #return df_group
    
    # Find all the areas where the events start and end
    # Reminder: Events start when the backward time difference is more than the cut-off (default 15)
    # Exclude rows where there is a start and end and the following row has a start
    # Also exclude rows where the duration is more than the cut-off (default 15) and the following row has a start
    #
    # The strict parameter includes an extra condition to consider points where the duration is more than minutes parameter
    # Strict is mainly relevant when the sampling frequency (e.g. 10 minutes) is large relative to the event cut-off (e.g. 15 minutes)
    # Example use-case for "strict":
    # 9:10 - 95.34
    # 9:20 - 95.36
    # 9:30 - 95.39
    # 9:40 - 95.39
    # 9:50 - 95.42
    # 10:00 - 95.42
    # 10:10 - 95.42
    # In this case, it is unclear whether the increase from 95.39 to 95.42 happened between 9:40 to 9:45 or between 9:45 to 9:50
    # If strict == on, this would be considered as 2 events (event 1: start = 9:10 and end = 9:30, event 2: start = 9:40 and end = 9:50) 
    # If strict == off, this would be considered as 1 event (event 1: start = 9:10 and end = 9:50)
    if (strict == "on"):
        df_event_start = df_group.loc[((df_group["time_diff_b"] >= minutes) & (df_group["time_diff_f"] < minutes)) |
                                      ((df_group["time_diff_b"] >= minutes) & (df_group["time_diff_f"] >= minutes) & (df_group["time_diff_b"].shift(-1) < minutes)) |
                                      ((df_group["duration"] >= minutes) & (df_group["time_diff_b"].shift(-1) < minutes)) |
                                      (((df_group["time_diff_b"] + df_group["duration"]) >= minutes) & ((df_group["time_diff_f"] + df_group["duration"]) >= minutes) & (df_group["time_diff_b"].shift(-1) < minutes))
                                     ].loc[:, ['account_id', 'measurementTime_max', 'lifetimeEnergy']]
    else:
        df_event_start = df_group.loc[((df_group["time_diff_b"] >= minutes) & (df_group["time_diff_f"] < minutes)) |
                                      ((df_group["time_diff_b"] >= minutes) & (df_group["time_diff_f"] >= minutes) & (df_group["time_diff_b"].shift(-1) < minutes)) |
                                      ((df_group["duration"] >= minutes) & (df_group["time_diff_b"].shift(-1) < minutes))
                                     ].loc[:, ['account_id', 'measurementTime_max', 'lifetimeEnergy']]
    
    # rename the columns
    df_event_start = df_event_start.rename(columns = {"measurementTime_max": "event_time_start",
                                                      "lifetimeEnergy": "event_energy_start"
                                                     })
    # do the same for the event ends
    if (strict == "on"):
        df_event_end = df_group.loc[((df_group["time_diff_b"] < minutes) & (df_group["time_diff_f"] >= minutes)) |
                                    ((df_group["time_diff_b"] >= minutes) & (df_group["time_diff_f"] >= minutes) & (df_group["time_diff_f"].shift(1) < minutes)) | 
                                    ((df_group["duration"] >= minutes) & (df_group["time_diff_f"].shift(1) < minutes)) |
                                    (((df_group["time_diff_b"] + df_group["duration"]) >= minutes) & ((df_group["time_diff_f"] + df_group["duration"]) >= minutes) & (df_group["time_diff_f"].shift(1) < minutes))
                                   ].loc[:, ['measurementTime_min', 'lifetimeEnergy']]
    else:
        df_event_end = df_group.loc[((df_group["time_diff_b"] < minutes) & (df_group["time_diff_f"] >= minutes)) |
                                    ((df_group["time_diff_b"] >= minutes) & (df_group["time_diff_f"] >= minutes) & (df_group["time_diff_f"].shift(1) < minutes)) | 
                                    ((df_group["duration"] >= minutes) & (df_group["time_diff_f"].shift(1) < minutes))
                                   ].loc[:, ['measurementTime_min', 'lifetimeEnergy']]
        
    df_event_end = df_event_end.rename(columns = {"measurementTime_min": "event_time_end",
                                                  "lifetimeEnergy": "event_energy_end"
                                                     })
    
    # add these into a new dataframe, drop the indexes
    df_events = pd.concat([df_event_start.reset_index(drop = True), df_event_end.reset_index(drop = True)], axis = 1)
    
    # add two new columns: event energy and event time and calculate the values
    df_events.insert(loc = len(df_events.columns), column = "event_energy", value = df_events["event_energy_end"] - df_events["event_energy_start"])
    df_events.insert(loc = len(df_events.columns), column = "event_time", value = df_events["event_time_end"] - df_events["event_time_start"])
    df_events.insert(loc = len(df_events.columns), column = "event_time_h", value = df_events["event_time"] / timedelta(hours=1))
    df_events.insert(loc = len(df_events.columns), column = "event_power_kW", value = df_events["event_energy"] / df_events["event_time_h"])
    
    
    clear_output(wait=True)
    return df_events

In [ ]:
def data_quality_check(df_i):
    """
    A tool for checking quality of data based on the event grouping
    df_events: the processed data set with the events
    """
    clear_output(wait=True)
    print("Checking quality...")
    df = df_i.copy()
    
    # Find the difference between the energy recorded at the end of the event and the energy recorded at the start of the next
    df["unaccounted_energy"] = df["event_energy_start"].shift(-1) - df["event_energy_end"]
    df["unaccounted_energy"].loc[df["account_id"] != df["account_id"].shift(-1)] = 0
    
    # Aggregate the data. Find the minimum and maximum recorded event energy for each meter and the sum of the missing energy
    df_q = df.groupby(["account_id"], as_index=False).aggregate(energy_min = ("event_energy_start", 'min'),
                                                                energy_max = ("event_energy_end", 'max'),
                                                                unaccounted_energy = ("unaccounted_energy", 'sum')
                                                               )
    
    # Compare the missing energy to the total energy
    df_q["quality"] = 1 - (df_q["unaccounted_energy"] / (df_q["energy_max"] - df_q["energy_min"]))
    df_q["quality"] = df_q["quality"].round(2)
    
    recorded_energy_total = round(df_q["energy_max"].sum() - df_q["energy_min"].sum(), 2)
    unaccounted_energy_total = round(df_q["unaccounted_energy"].sum(), 2)
    q_tot = round(100 - (unaccounted_energy_total * 100 / recorded_energy_total), 2)
    
    clear_output(wait=True)
    print("Total recorded energy = "+str(recorded_energy_total) + ". Unaccounted energy = " + str(unaccounted_energy_total) + ". " + str(q_tot) + "% of energy accounted for in events.")
    return df_q

In [ ]:
def data_quality_control(df_i, mode = "split", limit = 1.0):
    """
    A simple script for correcting the event definitions. Due to the way the meters function and are used in the field,
    there is often energy that occurs between the meter's save points. This is because the meters are continuously
    counting energ but only periodically save datapoints.
    
    When a meter is powered off (very common in cooking projects), the meter will have counted the energy used until 
    it was powered off but may not have saved the count as a datapoint. When the meter is powered back on, it resumes counting
    from where it left off but still may take several minutes to save its count as a new datapoint. Thus the energy count
    is accurate, but the timing of the usage is unclear.
    
    Example: An appliance uses 600W of continuous power and is connected to a meter that saves data every 10 mintues.
    At 9:00AM, the meter count is 200.00 kWh. The meter saves this datapoint.
    At 9:06AM, the meter count is 200.06 kWh. The meter is powered off.
    At 4:00:01PM, the meter is reconnected to power. The meter count resumes at 200.06 kWh.
    At 4:10PM, the meter count is 200.16 kWh. The meter saves this datapoint. 
    The datapoints will read:
    9:00AM: 200.00 kWh
    4:10PM: 200.16 kWh
    Thus the data shows an energy increase of 0.16 happening sometime during a 7 hour period. When processing this example
    as an event, one event would end at 9:00 AM and 200.00 kWh (the last recorded datapoint before a long gap) and the next
    event would start at 4:10PM and 200.16kWh (the first recorded datapoint after a long gap).
    This script corrects this by adding the 0.16 kWh of unaccounted energy back into the events. It also adjusts the times
    accordingly.
    
    Four modes of correction:
        conservative: the unaccounted energy is not included in any events (nothing happens!)
        trailing: the unaccounted energy is added to the preceding event (meter counts after saving)
        leading: the unaccounted energy is added to the following event (meter counts before saving)
        split: the unaccounted energy is split evenly between the preceding and following events
        
    limit:     energy gaps larger than the limit will be ignored and not corrected
    
    """
    df = df_i.copy()
    print("Events corrected. Mode = " + mode)
    
    if mode == "conservative":
        return df
    
    # Find the difference between the energy recorded at the end of the event and the energy recorded at the start of the next
    df["unaccounted_energy"] = df["event_energy_start"].shift(-1) - df["event_energy_end"]
    df["unaccounted_energy"].loc[df["account_id"] != df["account_id"].shift(-1)] = 0
    df["unaccounted_energy"].loc[df["unaccounted_energy"] > limit] = 0
    
    # Add a column for calculating the power in each event
    df["power"] = df["event_energy"] / (df["event_time"] / timedelta(hours=1))
    
    if mode == "trailing":
        df["event_energy_end"] = df["event_energy_end"] + df["unaccounted_energy"]
        df["event_time_end"] = df["event_time_end"] + ((df["unaccounted_energy"] / df["power"]) * timedelta(hours=1))
    elif mode == "leading":
        df["event_energy_start"] = df["event_energy_start"] - df["unaccounted_energy"].shift(1, fill_value = 0)
        df["event_time_start"] = df["event_time_start"] - ((df["unaccounted_energy"].shift(1, fill_value = 0) / df["power"]) * timedelta(hours=1))
    else:
        df["event_energy_end"] = df["event_energy_end"] + (df["unaccounted_energy"] / 2)
        df["event_energy_start"] = df["event_energy_start"] - (df["unaccounted_energy"].shift(1, fill_value = 0) / 2)
        
        df["event_time_end"] = df["event_time_end"] + (((df["unaccounted_energy"] / 2) / df["power"]) * timedelta(hours=1))
        df["event_time_start"] = df["event_time_start"] - (((df["unaccounted_energy"].shift(1, fill_value = 0) / 2) / df["power"]) * timedelta(hours=1))
    
    
    df["event_energy"] = df["event_energy_end"] - df["event_energy_start"]
    df["event_time_start"] = df["event_time_start"].dt.round("min")
    df["event_time_end"] = df["event_time_end"].dt.round("min")
    df["event_time"] = df["event_time_end"] - df["event_time_start"]
    
    #df["power_2"] = df["event_energy"] / (df["event_time"] / timedelta(hours=1))
    df = df.drop(columns = ["unaccounted_energy", "power"])
    
    return df

In [ ]:
def adjust_time(df_i, hours = 0):
    df = df_i.copy()
    df["event_time_start"] = df["event_time_start"] + timedelta(hours = hours)
    df["event_time_end"] = df["event_time_end"] + timedelta(hours = hours)
    return df

In [ ]:
def add_meterIds(df_i, df_lookup):
    """
    For adding the meterIds back into the events list
    Both dataframes must have columns named meterId and account_id
    """
    df = df_i.copy()
    df.insert(loc = 1, column = "meterId", value = df["account_id"])
    
    df["meterId"] = df["meterId"].replace(dict(zip(df_lookup["account_id"], df_lookup["meterId"])))
    return df

In [ ]:
def add_otherCol(df_i, df_lookup, col):
    """
    For adding the meterIds back into the events list
    Both dataframes must have columns named meterId and account_id
    """
    df = df_i.copy()
    df.insert(loc = 1, column = col, value = df["account_id"])
    
    df[col] = df[col].replace(dict(zip(df_lookup["account_id"], df_lookup[col])))
    return df

In [ ]:
def filter_events(df_i, filter_type = 'time', condition = 'below', filter_val = '00:01:00'):
    """
    Filter out the events based on criteria provided.
    
    filter_type should be either:
    1. time
    2. energy
    3. power
    
    condition should be either:
    1. above
    2. below
    """
    
    df = df_i.copy()
    
    if filter_type == 'time':
        col = "event_time"
    elif filter_type == 'energy':
        col = 'event_energy'
        filter_val = float(filter_val)
    elif filter_type == 'power':
        col = 'event_power'
        filter_val = float(filter_val)
        if col not in df.columns:
            df.insert(loc = len(df.columns), column = col, value = df["event_energy"] / (df["event_time"] / timedelta(hours=1)))
    elif filter_type == 'date':
        col = 'event_time_start'
    
    
    if condition == 'below':
        df_out = df.loc[df[col] > filter_val]
    elif condition == 'above':
        df_out = df.loc[df[col] < filter_val]
    
    return df_out

# Try it out

In [ ]:
# CHANGE THE BELOW to add the file path to the raw data
input_file_path = r"C:\Users\Lenovo\Documents\raw_data.csv"
raw_data = pd.read_csv(input_file_path, sep=',')

# Add the file path to the ID list (must have columns named meterId and account_id)
input_file_path = r"C:\Users\Lenovo\Documents\id_list.csv"
id_list = pd.read_csv(input_file_path, sep=',')

In [ ]:
time_start = time.time()

# Reduce the raw data
raw_data_r = reduce_columns(raw_data)
raw_data_g = group_data(raw_data_r)
raw_data_s = smooth_data(raw_data_g)

# Process the events list. Adjust the "minutes" parameter to suit the project needs. 
events_list = calculate_events(raw_data_s, minutes = 15)

# Small correction scripts. Use data_quality_control(events_list, mode="conservative") to minimize the effects.
data_quality = data_quality_check(events_list)
events_list_q = data_quality_control(events_list, limit = 0.3)
data_quality_q = data_quality_check(events_list_q)
# The unaccounted energy will be the energy that is not included in any events (energy gaps greater than the limit)

# Adjust the time
events_list_t = adjust_time(events_list_q, hours = 3)
events_list_f = add_meterIds(events_list_t, id_list)
raw_data_r_f = add_meterIds(raw_data_r, id_list)
time_end = time.time()
print("Complete. Total time = " + str(time_end - time_start))

In [ ]:
# Filter the events
# Adjust the parameters as necessary.
events_list_f_f = filter_events(events_list_f, filter_type = 'time', condition = 'below', filter_val = "00:06:00")
events_list_f_f = filter_events(events_list_f_f, filter_type = 'energy', condition = 'below', filter_val = "0.02")
events_list_f_f = filter_events(events_list_f_f, filter_type = 'power', condition = 'below', filter_val = "0.2")
events_list_f_f = filter_events(events_list_f_f, filter_type = 'power', condition = 'above', filter_val = "2.5")

In [ ]:
# save the filtered output
events_list_f_f.to_csv(r"C:\Users\Lenovo\Documents\example_f_f.csv")